# Lab 7 - st121413


1. Reproduce the vanilla GAN and DCGAN results on MNIST and CIFAR. Get the training and test loss for the generator and discriminator over time, plot them, and interpret them.

2. Develop your own GAN to model data generated as follows:
You should create a PyTorch DataSet that generates the 2D data in the __init__() method, outputs a sample in the __getitem__() method, and returns the dataset size in the __len__() method. Use the vanilla GAN approach above with an appropriate structure for the generator. Can your GAN generate a convincing facsimile of a set of samples from the actual distribution?


3. Use the DCGAN (or an improvement to it) to build a generator for a face image set of your choice. Can you get realistic faces that are not in the training set?


## 1. Reproduce the VanillaGAN and DGAN

The proof of reproducing the lab manual can be seen from the series of images from TensorboardX below.

### VGAN - MNIST
Loss:
Discriminator Loss is growning while the Generator loss is decreasing.

This could be that the objective of Discriminator is trying to classify a real data from fake one. Once the generator gets better at faking the data, a Discriminator could not classify the real fron the fake anymore.

The fake data generated at iteration 27,800 is also shown below.

![alt text](./screenshot/VGAN-lost.png)

![alt text](./screenshot/VGAN-result.png)


### DCGAN - CIFAR10

My DCGAN has totally different story from VanillaGAN.

The losses of Discriminator is all over the place. There is one iteration that the loss is very high and after that is alternate between super high and super low (0.9 or 0.0001)

The same case is applied to Generator Loss. It is reduced until that iteration and start to jump around.

This could the unstable problem we talked about in WGAN.

![alt text](./screenshot/DCGAN-loss.png)

![alt text](./screenshot/DCGAN-result.png)

## 2. VanillaGAN on SNAKE

First, here is my snake dataset

In [ ]:
import torch
class Snake(torch.utils.data.Dataset):
    def __init__(self, size = 100000):
        import numpy as np
        theta = torch.rand(size) * 2 * np.pi
        # torch.RAND
        # torch.rand(*size, *, out=None, dtype=None, layout=np.strided, device=None, requires_grad=False) → Tensor
        # Returns a tensor filled with random numbers from a uniform distribution on the interval [0, 1)[0,1)
        r = torch.randn(size)
        # np.RANDN
        # np.randn(*size, *, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) → Tensor
        # Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1 (also called the standard normal distribution).
        self.size = size
        self.x = (10 + r) * torch.cos(theta)
        mask = (theta >= (0.5 * np.pi)) & (theta <= (1.5 * np.pi))
        offset = torch.where(mask, torch.tensor(10.0), torch.tensor(-10.0))
        self.y = (10 + r) * torch.sin(theta) + offset

        self.x = torch.reshape(self.x, (1,-1))
        self.y = torch.reshape(self.y, (1,-1))
        self.data = torch.cat([ self.x, self.y], 0 ).T

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.size

Second, about fixing VanillaGAN.

At first, I change the number of <code>in feature</code> of Discriminator to 2 and Generator <code>out feature</code> to 2.

I ran it for a while and observed that my Generator can only output a certain range of number. 

I analyzed the Generator and decided to remove the last <code>tanh</code>. This function clipped out the output and I don't want that.

After doing so, the training goes well.

This is the setup where the <code>z</code> (input noise) space is 100-dimension. 

Later on, I found out from Chichan that I suppose to use <code>z</code> space of 2-dimension. 

Therefore, I changed the noise function to generate noise of size 2 and generator to accept data of size 2.

The loss graph can be found below.


The green line: 100-dimension z

The grey line: 2-dimension z

![alt text](./screenshot/VGAN-loss-snake.png)

The last 2 results of generator

![alt text](./screenshot/VGAN-result-snake.png)

## 3. DCGAN on Face (CelebA)

I choosed to use torchvision dataset named CelebA because it already there in the library.

The downloading procedure has some issue but I managed to make it working.

I modify not thing from DCGAN and just run the training because I want to get this result as a baseline.

My docker decided to not cooperate and vanished multiple times.

At the time of writing, my model is still training very slowly.

I share you my current result.

![alt text](./screenshot/DCGAN-loss-face.png)

![alt text](./screenshot/DCGAN-result-face.png)

From my analysis, I think the dataset is not only contain faces but rather an image of Celebrity with poses. The varity is just to big for this small Generator to learn.

Since the Generator failed to output the image, the discriminator has en easy time seperate real data from fake data.

### update 6/03/2021

I have trained DCGAN on face again using ait-ict.zip given for the cycle gan the reason is

1. The generator we have does not has a lot of weight to adjust.
2. This ait-ict dataset has only 300 images with better face alignment.

I strongly believe that this will allow my generator to learn.


## Experiment 1 - DCGAN + ICT face

I ran about 300 epochs and observed that the generator learned much better than the CelebA. However, the face does not clear. I guess it is due to 3 channels. Therefore, I decide to stop the train.

![alt text](./screenshot/DCGAN-ICT-loss.png)

![alt text](./screenshot/DCGAN-ICT-result.png)


## Experiment 2 - DCGAN + ICT grey face

I convert the images to 1 channel with resize to 64x64. Then I modify DCGAN to output an images of 1 channel (Generator) and classify on 1 channel image (Discriminator)

As shown in the firgures below, my GAN doing great until at one epoch it turned into mode collapse and onward.

![alt text](./screenshot/DCGAN-ICT-grey-loss.png)

![alt text](./screenshot/DCGAN-ICT-grey-1.png)

![alt text](./screenshot/DCGAN-ICT-grey-2.png)

![alt text](./screenshot/DCGAN-ICT-grey-3.png)